In [1]:
import os
import sys; sys.path.append("~/HHbbgg_ETH/Training/python") # to load packages
import training_utils as utils
import numpy as np
reload(utils)

Welcome to ROOTaaS 6.05/02


<module 'training_utils' from '/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/Training/python/training_utils.pyc'>

In [2]:
reload(utils)
ntuples = '20170620'
# "%" sign allows to interpret the rest as a system command
%env data=$utils.IO.ldata$ntuples
files = ! ls $data | sort -t_ -k 3 -n
#pick only SM
signal = [s for s in files if "GluGluToHHTo2B2G_node_SM" in s]
diphotonJets = [s for s in files if "DiPhotonJets" in s]
gJets_lowPt = [s for s in files if "GJet_Pt-20to40" in s]
gJets_highPt = [s for s in files if "GJet_Pt-40" in s]
ggH = [s for s in files if "GluGluHToGG" in s]
vbf = [s for s in files if "VBFHToGG" in s]
VH = [s for s in files if "VHToGG" in s]
bbH = [s for s in files if "bbHToGG_M-125_4FS_yb2" in s] #is this correct?
ttH = [s for s in files if "ttHToGG" in s]


utils.IO.add_signal(ntuples,signal,1)
utils.IO.add_background(ntuples,diphotonJets,-1)
utils.IO.add_background(ntuples,gJets_lowPt,-2)
utils.IO.add_background(ntuples,gJets_highPt,-2)
utils.IO.add_background(ntuples,ggH,-3)
utils.IO.add_background(ntuples,vbf,-4)
utils.IO.add_background(ntuples,VH,-5)
utils.IO.add_background(ntuples,bbH,-6)
utils.IO.add_background(ntuples,ttH,-7)



for i in range(len(utils.IO.backgroundName)):        
    print "using background file n."+str(i)+": "+utils.IO.backgroundName[i]
for i in range(len(utils.IO.signalName)):    
    print "using signal file n."+str(i)+": "+utils.IO.signalName[i]

env: data=/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620
using background file n.0: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa.root
using background file n.1: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GJet_Pt-20to40_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8.root
using background file n.2: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GJet_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8.root
using background file n.3: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GluGluHToGG_M-125_13TeV_powheg_pythia8.root
using background file n.4: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_VBFHToGG_M-125_13TeV_powheg_pythia8.root
using background file n.5: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_VHToGG_M125_13TeV_amcatnloFXFX_madspin_pythia8.

In [3]:
#use noexpand for root expressions, it needs this file https://github.com/ibab/root_pandas/blob/master/root_pandas/readwrite.py
#all variables
branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M(),customLeadingPhotonIDMVA,customSubLeadingPhotonIDMVA,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverMDecorr,DiJetDiPho_DR'.split(",")
#std variables
#branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M()'.split(",")
branch_names = [c.strip() for c in branch_names]
#branch_names = (b.replace(" ", "_") for b in branch_names)
#branch_names = list(b.replace("-", "_") for b in branch_names)
print branch_names

import pandas as pd
import root_pandas as rpd

#utils.IO.get_cross_sections('jsons/datasets80X_Moriond.json')
#utils.IO.w_sig.append(utils.IO.get_weight('signal','nrSM',0))
#utils.IO.w_bkg(utils.IO.get_weight('background','DiPhoJets',0))
#utils.IO.w_bkg(utils.IO.get_weight('background','GJets',0))
#utils.IO.w_bkg(utils.IO.get_weight('background','GJets',1))

####FIXME!!! shuffle for background needed
utils.preprocessing.set_signals_and_backgrounds("bbggSelectionTree",branch_names,shuffle=False)
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=utils.preprocessing.set_variables(branch_names)

#relative weighting between components of one class is kept, all classes normalized to the same
#weights_sig=utils.preprocessing.weight_signal_with_resolution(weights_sig,y_sig)

#X_bkg,y_bkg,weights_bkg = utils.preprocessing.randomize(X_bkg,y_bkg,weights_bkg)
#X_sig,y_sig,weights_sig = utils.preprocessing.randomize(X_sig,y_sig,weights_sig)


#bbggTrees have by default signal and CR events, let's be sure that we clean it
#FIXME! just for debugging!!!!!!! uncomment
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=utils.preprocessing.clean_signal_events(X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig)


#y_total_train = utils.preprocessing.get_total_training_sample(y_sig,y_bkg).ravel()
#X_total_train = utils.preprocessing.get_total_training_sample(X_sig,X_bkg)

#y_total_test = utils.preprocessing.get_total_test_sample(y_sig,y_bkg).ravel()
#X_total_test = utils.preprocessing.get_total_test_sample(X_sig,X_bkg)

#w_total_train = utils.preprocessing.get_total_training_sample(weights_sig,weights_bkg).ravel()
#w_total_test = utils.preprocessing.get_total_test_sample(weights_sig,weights_bkg).ravel()
#np.savetxt('/tmp/micheli/xsig_3.txt',X_sig)
#utils.preprocessing.adjust_and_compress(utils.IO.signal_df[0]).to_csv('/tmp/micheli/signal_3.csv') 

np.savetxt('/tmp/micheli/xbkg.txt',X_bkg)
utils.preprocessing.adjust_and_compress(utils.IO.background_df[0]).to_csv('/tmp/micheli/bkg.csv')

print X_sig.shape
print X_sig[0]

['leadingJet_bDis', 'subleadingJet_bDis', 'noexpand:fabs(CosThetaStar_CS)', 'noexpand:fabs(CosTheta_bb)', 'noexpand:fabs(CosTheta_gg)', 'noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M()', 'noexpand:dijetCandidate.Pt()/diHiggsCandidate.M()', 'customLeadingPhotonIDMVA', 'customSubLeadingPhotonIDMVA', 'leadingPhotonSigOverE', 'subleadingPhotonSigOverE', 'sigmaMOverMDecorr', 'DiJetDiPho_DR']
(18646, 13)
[ 0.989  0.065  0.152  0.482  0.223  0.402  0.551  0.967  0.862  0.005
  0.007  0.006  2.261]


In [4]:
# load the model from disk
from sklearn.externals import joblib
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH/Training/output_files/allMC_stdVariables.pkl'))
loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH/Training/output_files/allMC_resWeighting_optimized.pkl'))
#sig = np.concatenate((X_total_train[y_total_train > 0],X_total_test[y_total_test > 0]))
#bkg_0 = np.concatenate((X_total_train[y_total_train == -1],X_total_test[y_total_test == -1]))
bkg_0 = X_bkg[y_bkg ==-1]
bkg_1 = X_bkg[y_bkg ==-2]
bkg_3 = X_bkg[y_bkg ==-3]
bkg_4 = X_bkg[y_bkg ==-4]
bkg_5 = X_bkg[y_bkg ==-5]
bkg_6 = X_bkg[y_bkg ==-6]
bkg_7 = X_bkg[y_bkg ==-7]



#bkg_1 = np.concatenate((X_total_train[y_total_train == -2],X_total_test[y_total_test == -2]))
#bkg_3 = np.concatenate((X_total_train[y_total_train == -3],X_total_test[y_total_test == -3]))
#bkg_4 = np.concatenate((X_total_train[y_total_train == -4],X_total_test[y_total_test == -4]))
#bkg_5 = np.concatenate((X_total_train[y_total_train == -5],X_total_test[y_total_test == -5]))
#bkg_6 = np.concatenate((X_total_train[y_total_train == -6],X_total_test[y_total_test == -6]))
#bkg_7 = np.concatenate((X_total_train[y_total_train == -7],X_total_test[y_total_test == -7]))


Y_pred_sig = loaded_model.predict_proba(X_sig)[:,2]
Y_pred_bkg_0 = loaded_model.predict_proba(bkg_0)[:,2]
Y_pred_bkg_1 = loaded_model.predict_proba(bkg_1)[:,2]
Y_pred_bkg_3 = loaded_model.predict_proba(bkg_3)[:,2]
Y_pred_bkg_4 = loaded_model.predict_proba(bkg_4)[:,2]
Y_pred_bkg_5 = loaded_model.predict_proba(bkg_5)[:,2]
Y_pred_bkg_6 = loaded_model.predict_proba(bkg_6)[:,2]
Y_pred_bkg_7 = loaded_model.predict_proba(bkg_7)[:,2]

print y_sig.shape
print Y_pred_sig.shape
print Y_pred_bkg_0.shape
print Y_pred_bkg_1.shape
print utils.IO.signal_df[0].index.shape
#np.savetxt('/tmp/micheli/prova2.txt',bkg_6)
np.savetxt('/tmp/micheli/prova8.txt',Y_pred_sig)
#v = np.column_stack((utils.IO.signal_df[0].index,Y_pred_sig))
#print v
#np.savetxt('/tmp/micheli/prova8_2.txt',v)
#additionalCut_names = 'noexpand:diphotonCandidate.M(),noexpand:dijetCandidate.M()'.split(",")
additionalCut_names = 'noexpand:diphotonCandidate.M(),noexpand:dijetCandidate.M()'.split(",")
sig_count_df = rpd.read_root(utils.IO.signalName[0],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(sig_count_df,utils.IO.sigProc[0],utils.IO.signalName[0])

#print utils.IO.signal_df[0][['random_index']]
#sig_count_df['random_index']=utils.IO.signal_df[0][['random_index']]
#sig_count_df.sort_values(by='random_index',inplace=True)

(18646,)
(18646,)
(1900684,)
(22278,)
(21184,)


In [5]:
#### here counting events only in dijet mass window######
#print utils.IO.signal_df[0]['proc']
#print utils.IO.signal_df[0].index
print sig_count_df['diphotonCandidate.M()'].shape
M_sig = np.round(np.asarray(sig_count_df['diphotonCandidate.M()']),6)
print 'M_sig.shape'
print M_sig.shape
W_sig = np.asarray(sig_count_df['weight'][utils.IO.signal_df[0].index])
M_dijet_sig = np.round(np.asarray(sig_count_df['dijetCandidate.M()']),6)
nTot = np.column_stack((M_sig,W_sig,M_dijet_sig))
print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,1]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
print "Y_pred"
print Y_pred_sig.shape
nCleaned_MVA = nCleaned[np.where(Y_pred_sig>0.40)]
#apply massWindow +- 1.5
print nCleaned_MVA.shape
print np.where(nCleaned_MVA[:,0]>123.5000)
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5000)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5000)]
print "mass window"

print nCleaned_massWindowDown.shape
print nCleaned_massWindow.shape
print nCleaned_massWindow[:,1]
nEvts = sum(nCleaned_massWindow[:,1])
print "signal events with 36 fb-1: "+str(nEvts)
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass:"+str(nEvts_Cut)
np.savetxt('/tmp/micheli/nCleaned_massWindowDown_3.txt',nCleaned_massWindowDown)
np.savetxt('/tmp/micheli/nCleaned_MVA.txt',np.column_stack((nCleaned_MVA,Y_pred_sig[np.where(Y_pred_sig>0.40)])))

(21184,)
M_sig.shape
(21184,)
(21184, 3)
(18646, 3)
Y_pred
(18646,)
(17097, 3)
(array([    0,     2,     3, ..., 17093, 17095, 17096]),)
mass window
(13995, 3)
(13284, 3)
[ 0.0007174  0.0007174  0.0007174 ...,  0.0007174  0.0007174  0.0007174]
signal events with 36 fb-1: 9.5299416
(8412, 3)
after cutting in dijet mass:6.0347688


In [6]:
bkg_3_count_df = rpd.read_root(utils.IO.backgroundName[0],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_3_count_df,utils.IO.bkgProc[0],utils.IO.backgroundName[0])

M_bkg = np.asarray(bkg_3_count_df['diphotonCandidate.M()'][utils.IO.background_df[0].index])
W_bkg = np.asarray(bkg_3_count_df['weight'][utils.IO.background_df[0].index])
M_dijet_bkg = np.asarray(bkg_3_count_df['dijetCandidate.M()'][utils.IO.background_df[0].index])
nTot = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))

print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,1]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_0>0.40)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,1])
print utils.IO.backgroundName[0].split("/")[len(utils.IO.backgroundName[0].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
print "with scale factor:"+str(nEvts*1.5) 
Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass (with scale factor):"+str(nEvts_Cut*1.5)

(2190915, 3)
(1900684, 3)
(268554, 3)
(12234, 3)
DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa events with 36 fb-1: 209.944387432
with scale factor:314.916581148
(3790, 3)
after cutting in dijet mass (with scale factor):95.136941173


In [7]:
bkg_1_count_df = rpd.read_root(utils.IO.backgroundName[1],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_1_count_df,utils.IO.bkgProc[1],utils.IO.backgroundName[1])

M_bkg = np.asarray(bkg_1_count_df['diphotonCandidate.M()'][utils.IO.background_df[1].index])
crazySF_20=25
W_bkg = np.asarray(np.multiply(bkg_1_count_df['weight'][utils.IO.background_df[1].index],crazySF_20))
M_dijet_bkg = np.asarray(bkg_1_count_df['dijetCandidate.M()'][utils.IO.background_df[1].index])

nTot = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))
print nTot.shape

bkg_2_count_df = rpd.read_root(utils.IO.backgroundName[2],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_2_count_df,utils.IO.bkgProc[2],utils.IO.backgroundName[2])

M_bkg = np.asarray(bkg_2_count_df['diphotonCandidate.M()'][utils.IO.background_df[2].index])
crazySF_40=3
W_bkg = np.asarray(np.multiply(bkg_2_count_df['weight'][utils.IO.background_df[2].index],crazySF_40))
M_dijet_bkg = np.asarray(bkg_2_count_df['dijetCandidate.M()'][utils.IO.background_df[2].index])

nTot_2 = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))

nTot_3 = np.concatenate((nTot,nTot_2))

print nTot_3.shape
#apply isSignal cleaning
nCleaned = nTot_3[np.where(nTot_3[:,1]!=0),:][0]
print "nCleaned"
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_1>0.40)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,1])
print utils.IO.backgroundName[1].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass:"+str(nEvts_Cut)

#np.savetxt('/tmp/micheli/prova7.txt',nCleaned_mdijet)
#utils.preprocessing.adjust_and_compress(bkg_2_count_df).to_csv('/tmp/micheli/background_7_2.csv') 

(2590, 3)
(63259, 3)
nCleaned
(22278, 3)
(608, 3)
(27, 3)
GJet_Pt-20to40_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8 events with 36 fb-1: 40.8667117897
(5, 3)
after cutting in dijet mass:6.34077751559


In [8]:
bkg_3_count_df = rpd.read_root(utils.IO.backgroundName[3],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_3_count_df,utils.IO.bkgProc[3],utils.IO.backgroundName[3])

M_bkg = np.asarray(bkg_3_count_df['diphotonCandidate.M()'][utils.IO.background_df[3].index])
W_bkg = np.asarray(bkg_3_count_df['weight'][utils.IO.background_df[3].index])
M_dijet_bkg = np.asarray(bkg_3_count_df['dijetCandidate.M()'][utils.IO.background_df[3].index])
nTot = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))



print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,1]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_3>0.40)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,1])
print utils.IO.backgroundName[3].split("/")[len(utils.IO.backgroundName[3].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass:"+str(nEvts_Cut)

(63320, 3)
(56382, 3)
(10430, 3)
(8429, 3)
GluGluHToGG_M-125_13TeV_powheg_pythia8 events with 36 fb-1: 32.1702115094
(2456, 3)
after cutting in dijet mass:9.37359585563


In [9]:
bkg_4_count_df = rpd.read_root(utils.IO.backgroundName[4],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_4_count_df,utils.IO.bkgProc[4],utils.IO.backgroundName[4])

M_bkg = np.asarray(bkg_4_count_df['diphotonCandidate.M()'][utils.IO.background_df[4].index])
W_bkg = np.asarray(bkg_4_count_df['weight'][utils.IO.background_df[4].index])
M_dijet_bkg = np.asarray(bkg_4_count_df['dijetCandidate.M()'][utils.IO.background_df[4].index])
nTot = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))



print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,1]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_4>0.40)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,1])
print utils.IO.backgroundName[4].split("/")[len(utils.IO.backgroundName[4].split("/"))-1].replace(".root","").replace("output_","")+"  events with 36 fb-1: "+str(nEvts)
Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass:"+str(nEvts_Cut)

(84846, 3)
(76431, 3)
(17392, 3)
(13945, 3)
VBFHToGG_M-125_13TeV_powheg_pythia8  events with 36 fb-1: 4.5443142217
(3960, 3)
after cutting in dijet mass:1.29015496934


In [10]:
bkg_5_count_df = rpd.read_root(utils.IO.backgroundName[5],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_5_count_df,utils.IO.bkgProc[5],utils.IO.backgroundName[5])

M_bkg = np.asarray(bkg_5_count_df['diphotonCandidate.M()'][utils.IO.background_df[5].index])
W_bkg = np.asarray(bkg_5_count_df['weight'][utils.IO.background_df[5].index])
M_dijet_bkg = np.asarray(bkg_5_count_df['dijetCandidate.M()'][utils.IO.background_df[5].index])
nTot = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))



print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,1]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_5>0.40)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,1])
print utils.IO.backgroundName[5].split("/")[len(utils.IO.backgroundName[5].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass:"+str(nEvts_Cut)

(65684, 3)
(57455, 3)
(19426, 3)
(15313, 3)
VHToGG_M125_13TeV_amcatnloFXFX_madspin_pythia8 events with 36 fb-1: 8.85203301101
(4107, 3)
after cutting in dijet mass:2.16708525497


In [11]:
bkg_6_count_df = rpd.read_root(utils.IO.backgroundName[6],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_6_count_df,utils.IO.bkgProc[6],utils.IO.backgroundName[6])

M_bkg = np.asarray(bkg_6_count_df['diphotonCandidate.M()'][utils.IO.background_df[6].index])
W_bkg = np.asarray(bkg_6_count_df['weight'][utils.IO.background_df[6].index])
M_dijet_bkg = np.asarray(bkg_6_count_df['dijetCandidate.M()'][utils.IO.background_df[6].index])
nTot = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))



print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,1]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_6>0.40)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,1])
print utils.IO.backgroundName[6].split("/")[len(utils.IO.backgroundName[6].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass:"+str(nEvts_Cut)

(123341, 3)
(106395, 3)
(48183, 3)
(35935, 3)
bbHToGG_M-125_4FS_yb2_13TeV_amcatnlo events with 36 fb-1: 2.11172646383
(12668, 3)
after cutting in dijet mass:0.745007494012


In [12]:
bkg_7_count_df = rpd.read_root(utils.IO.backgroundName[7],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_7_count_df,utils.IO.bkgProc[7],utils.IO.backgroundName[7])

M_bkg = np.asarray(bkg_7_count_df['diphotonCandidate.M()'][utils.IO.background_df[7].index])
W_bkg = np.asarray(bkg_7_count_df['weight'][utils.IO.background_df[7].index])
M_dijet_bkg = np.asarray(bkg_7_count_df['dijetCandidate.M()'][utils.IO.background_df[7].index])
nTot = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))



print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,1]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_7>0.40)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,1])
print utils.IO.backgroundName[7].split("/")[len(utils.IO.backgroundName[7].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass:"+str(nEvts_Cut)

(350826, 3)
(268863, 3)
(239532, 3)
(180439, 3)
ttHToGG_M125_13TeV_powheg_pythia8_v2 events with 36 fb-1: 10.2747198463
(73048, 3)
after cutting in dijet mass:4.15950067579


In [13]:
print X_bkg[y_bkg == -4]

[[ 0.31   0.533  0.824 ...,  0.01   0.006  2.729]
 [ 0.149  0.16   0.981 ...,  0.007  0.006  2.407]
 [ 0.099  0.768  0.471 ...,  0.011  0.006  3.289]
 ..., 
 [ 0.833  0.527  0.594 ...,  0.01   0.007  2.8  ]
 [ 0.464  0.998  0.56  ...,  0.007  0.006  3.058]
 [ 0.091  0.175  0.936 ...,  0.007  0.01   2.69 ]]
